In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
bene_train = pd.read_csv('/content/Train_Beneficiarydata-1542865627584.csv')
bene_test= pd.read_csv('/content/Test_Beneficiarydata-1542969243754.csv')

inp_test = pd.read_csv('/content/Train_Inpatientdata-1542865627584.csv')
inp_train = pd.read_csv('/content/Test_Inpatientdata-1542969243754.csv')

outp_test= pd.read_csv('/content/Test_Outpatientdata-1542969243754.csv')
outp_train = pd.read_csv('/content/Train_Outpatientdata-1542865627584.csv')

label_train = pd.read_csv('/content/Train-1542865627584.csv')
label_test = pd.read_csv('/content/Test-1542969243754.csv')

In [3]:
bene_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

In [4]:
bene_train.shape, bene_test.shape

((138556, 25), (63968, 25))

In [5]:
bene_train.columns


Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt'],
      dtype='object')

In [6]:
bene_train['RenalDiseaseIndicator'].value_counts()

,count
RenalDiseaseIndicator,
0,118978
Y,19578


In [7]:
bene_train['RenalDiseaseIndicator'] = bene_train['RenalDiseaseIndicator'].replace('Y',1).astype(int)
bene_train['RenalDiseaseIndicator'].value_counts()

,count
RenalDiseaseIndicator,
0,118978
1,19578


In [8]:
for col in ['DOB','DOD']:
    bene_train[col] = pd.to_datetime(bene_train[col])


In [9]:
bene_train['BirthYear'] = bene_train['DOB'].dt.year
max_bene_DOD = max(bene_train['DOD'].unique()[1:])
max_bene_DOD

Timestamp('2009-12-01 00:00:00')

In [10]:
bene_train['DOD'].fillna(value=max_bene_DOD, inplace=True)
bene_train['Age'] = round(((bene_train['DOD'] - bene_train['DOB']).dt.days)/365,0)


In [11]:
bene_train['Age'].describe()

,Age
count,138556.000000
mean,73.622831
std,12.733335
min,26.000000
25%,68.000000
50%,74.000000
75%,82.000000
max,101.000000


In [12]:
bene_train['Alive'] = bene_train['DOD'].apply(lambda x: 1 if x != x else 0)
bene_train['Alive'].value_counts()

,count
Alive,
0,138556


In [13]:
bene_train.groupby('NoOfMonths_PartACov').count()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive
NoOfMonths_PartACov,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000,1000
1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
3,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
4,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
5,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8
6,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38,38
7,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16
8,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26


In [14]:
bene_train.groupby('NoOfMonths_PartBCov').count()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive
NoOfMonths_PartBCov,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675,675
1,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
2,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19,19
3,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27,27
4,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35,35
5,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50,50
6,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282,282
7,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63,63
8,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71,71


In [15]:
bene_train.columns

Index(['BeneID', 'DOB', 'DOD', 'Gender', 'Race', 'RenalDiseaseIndicator',
       'State', 'County', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'BirthYear', 'Age', 'Alive'],
      dtype='object')

In [16]:
diseases = ['ChronicCond_Alzheimer','ChronicCond_Heartfailure',
           'ChronicCond_KidneyDisease','ChronicCond_Cancer','ChronicCond_ObstrPulmonary',
          'ChronicCond_Depression','ChronicCond_Diabetes','ChronicCond_IschemicHeart','ChronicCond_Osteoporasis',
           'ChronicCond_rheumatoidarthritis','ChronicCond_stroke']

In [17]:
for disease in diseases:
    bene_train[disease] = bene_train[disease].replace({1:'Yes', 2: 'No'})

In [18]:
for i in range(bene_train.shape[0]):
    chronicDCnt = 0
    for disease in diseases:
        if bene_train.loc[i,disease] == 'Yes':
                chronicDCnt+=1
        else:
                continue
    bene_train.loc[i,'ChronicDisease_Count'] = chronicDCnt

In [19]:
bene_train.head(2)

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
0,BENE11001,1943-01-01,2009-12-01,1,1,0,39,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
1,BENE11002,1936-09-01,2009-12-01,2,1,0,39,280,12,12,No,No,No,No,No,No,No,No,No,No,No,0,0,30,50,1936,73.0,0,0.0


In [20]:
bene_train.shape

(138556, 29)

In [21]:
bene_train['Gender'].value_counts()

,count
Gender,
2,79106
1,59450


In [22]:
bene_train['Gender'] = bene_train['Gender'].replace({1:'Male', 2:'Female'})

In [23]:
bene_train['Race'] = bene_train['Race'].replace({1:'White', 2:'Black or African American',3:'Other', 5:'Hispanic'})

In [24]:
bene_train['Race'].value_counts()

,count
Race,
White,117057
Black or African American,13538
Other,5059
Hispanic,2902


In [25]:
state_mapping = {
    1: 'Alabama', 2: 'Alaska', 3: 'Arizona', 4: 'Arkansas', 5: 'California',
    6: 'Colorado', 7: 'Connecticut', 8: 'Delaware', 9: 'District of Columbia',
    10: 'Florida', 11: 'Georgia', 12: 'Hawaii', 13: 'Idaho', 14: 'Illinois',
    15: 'Indiana', 16: 'Iowa', 17: 'Kansas', 18: 'Kentucky', 19: 'Louisiana',
    20: 'Maine', 21: 'Maryland', 22: 'Massachusetts', 23: 'Michigan',
    24: 'Minnesota', 25: 'Mississippi', 26: 'Missouri', 27: 'Montana',
    28: 'Nebraska', 29: 'Nevada', 30: 'New Hampshire', 31: 'New Jersey',
    32: 'New Mexico', 33: 'New York', 34: 'North Carolina', 35: 'North Dakota',
    36: 'Ohio', 37: 'Oklahoma', 38: 'Oregon', 39: 'Pennsylvania',
    40: 'Rhode Island', 41: 'South Carolina', 42: 'South Dakota',
    43: 'Tennessee', 44: 'Texas', 45: 'Utah', 46: 'Vermont', 47: 'Virginia',
    48: 'Washington', 49: 'West Virginia', 50: 'Wisconsin', 51: 'Wyoming',
    52: 'American Samoa', 53: 'Guam', 54: 'Puerto Rico'
}

bene_train['State'] = bene_train['State'].replace(state_mapping)

In [26]:
outp_train.shape, inp_train.shape

((517737, 27), (9551, 30))

In [27]:
outp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,V5866,V1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,30000,72887,4280,7197,V4577,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [28]:
inp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,NaN,2009-09-09,5789,1068.0,2009-09-16,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,NaN,NaN,NaN,NaN
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,NaN,2008-12-25,5939,1068.0,2009-01-08,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,NaN,5551.0,NaN,NaN,NaN,NaN,NaN
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,NaN,NaN,2009-12-09,4019,1068.0,2009-12-13,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,NaN,2009-06-23,8208,1068.0,2009-07-06,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,NaN,NaN,NaN,NaN
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,NaN,NaN,2009-01-20,4279,1068.0,2009-01-31,880,29654,V142,78702,30503,V140,V4582,V6109,7242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
inp_train['ClaimStartDt'].min()

'2008-11-27'

In [30]:
for col in ['ClaimEndDt','ClaimStartDt', 'DischargeDt', 'AdmissionDt']:
    inp_train[col] = pd.to_datetime(inp_train[col])

for col in ['ClaimEndDt','ClaimStartDt']:
    outp_train[col] = pd.to_datetime(outp_train[col])

In [31]:
inp_train['ClaimPeriod'] =  round(((inp_train.ClaimEndDt - inp_train.ClaimStartDt).dt.days),0)
outp_train['ClaimPeriod'] =  round(((outp_train.ClaimEndDt - outp_train.ClaimStartDt).dt.days),0)

In [32]:
inp_train['TimeInHptal'] =  round(((inp_train.DischargeDt - inp_train.AdmissionDt).dt.days),0)

In [33]:
inp_train.drop(['DischargeDt','AdmissionDt'],axis=1,inplace=True)

In [35]:
ClmProcedureCode = ['ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3',
        'ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6',]

ClmDiagnosisCode = ['ClmDiagnosisCode_1',
       'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4',
       'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7',
       'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10']

In [36]:
for code in ClmDiagnosisCode:
    inp_train.fillna('-1',inplace=True)
    outp_train.fillna('-1',inplace=True)

def CountingIndex(df,codeList,indexName):

    for i in range(df.shape[0]):#
        DiagnosisCnt = 0
        for code in codeList:
            if df.loc[i,code]== '-1' :
                continue
            else:
                 DiagnosisCnt+=1
        df.loc[i,indexName] = DiagnosisCnt

In [37]:
CountingIndex(inp_train,ClmDiagnosisCode,'Diagnosis Count')
CountingIndex(outp_train,ClmDiagnosisCode,'Diagnosis Count')

In [38]:
CountingIndex(inp_train,ClmProcedureCode,'Procedures Count')
CountingIndex(outp_train,ClmProcedureCode,'Procedures Count')

In [39]:
inp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0


In [40]:
def isSamePhysician(df):
    for i in range(0,df.shape[0]):

        if  df.loc[i,['AttendingPhysician']][0] == df.loc[i,['OperatingPhysician']][0]:
            df.loc[i,'SamePhysician'] = 1
        else:
            df.loc[i,'SamePhysician'] = 0

In [45]:
inp_train.OtherPhysician.fillna(0,inplace=True)
outp_train.OtherPhysician.fillna(0,inplace=True)

In [47]:
inp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0,0.0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0,0.0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0,0.0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0,0.0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0,0.0


In [48]:
inp_train.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmAdmitDiagnosisCode', 'DeductibleAmtPaid',
       'DiagnosisGroupCode', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician'],
      dtype='object')

In [49]:
bene_train.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count
0,BENE11001,1943-01-01,2009-12-01,Male,White,0,Pennsylvania,230,12,12,Yes,No,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,36000,3204,60,70,1943,67.0,0,7.0
1,BENE11002,1936-09-01,2009-12-01,Female,White,0,Pennsylvania,280,12,12,No,No,No,No,No,No,No,No,No,No,No,0,0,30,50,1936,73.0,0,0.0
2,BENE11003,1936-08-01,2009-12-01,Male,White,0,American Samoa,590,12,12,Yes,No,No,No,No,No,No,Yes,No,No,No,0,0,90,40,1936,73.0,0,2.0
3,BENE11004,1922-07-01,2009-12-01,Male,White,0,Pennsylvania,270,12,12,Yes,Yes,No,No,No,No,Yes,Yes,Yes,Yes,No,0,0,1810,760,1922,87.0,0,6.0
4,BENE11005,1935-09-01,2009-12-01,Male,White,0,Minnesota,680,12,12,No,No,No,No,Yes,No,Yes,No,No,No,No,0,0,1790,1200,1935,74.0,0,2.0


In [50]:
inp_train.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmAdmitDiagnosisCode', 'DeductibleAmtPaid',
       'DiagnosisGroupCode', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician'],
      dtype='object')

In [51]:
inp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0,0.0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0,0.0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0,0.0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0,0.0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0,0.0


In [52]:
inp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician']

In [53]:
outp_train.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode', 'ClaimPeriod', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician'],
      dtype='object')

In [54]:
outp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician']

In [55]:
label_train.head()

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No


In [56]:
label_train.shape

(5410, 2)

In [57]:
label_train['PotentialFraud'].replace('No',0,inplace=True)
label_train['PotentialFraud'].replace('Yes',1,inplace=True)

fraud = label_train[label_train['PotentialFraud']==1]
fraud.shape[0]

506

In [58]:
inp_train['OPD_Flag'] = 0

In [59]:
inp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod',
       'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician', 'OPD_Flag']

In [60]:
inp_imp_cols

['BeneID',
 'ClaimID',
 'Provider',
 'InscClaimAmtReimbursed',
 'DeductibleAmtPaid',
 'ClaimPeriod',
 'TimeInHptal',
 'Diagnosis Count',
 'Procedures Count',
 'SamePhysician',
 'OPD_Flag']

In [62]:
outp_train['TimeInHptal'] = 0
outp_train['OPD_Flag'] = 1

In [61]:
outp_imp_cols = ['BeneID', 'ClaimID', 'Provider',
       'InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal','Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag']

In [63]:
outp_imp_cols

['BeneID',
 'ClaimID',
 'Provider',
 'InscClaimAmtReimbursed',
 'DeductibleAmtPaid',
 'ClaimPeriod',
 'TimeInHptal',
 'Diagnosis Count',
 'Procedures Count',
 'SamePhysician',
 'OPD_Flag']

In [64]:
inp_train.columns, outp_train.columns

(Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
        'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
        'OtherPhysician', 'ClmAdmitDiagnosisCode', 'DeductibleAmtPaid',
        'DiagnosisGroupCode', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
        'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
        'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
        'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
        'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
        'ClmProcedureCode_5', 'ClmProcedureCode_6', 'ClaimPeriod',
        'TimeInHptal', 'Diagnosis Count', 'Procedures Count', 'SamePhysician',
        'OPD_Flag'],
       dtype='object'),
 Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
        'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
        'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2'

In [65]:
outp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode,ClaimPeriod,Diagnosis Count,Procedures Count,SamePhysician,TimeInHptal,OPD_Flag
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,-1,-1,78943,V5866,V1272,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,56409,0,3.0,0.0,0.0,0,1
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,-1,-1,6115,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,79380,0,1.0,0.0,0.0,0,1
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,-1,-1,2723,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,1.0,0.0,0.0,0,1
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,-1,-1,71988,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,0,1.0,0.0,0.0,0,1
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,-1,-1,82382,30000,72887,4280,7197,V4577,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,71947,0,6.0,0.0,0.0,0,1


In [66]:
inp_train.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag
0,BENE11014,CLM67387,2009-09-09,2009-09-16,PRV57070,9000,PHY317786,PHY427017,-1,5789,1068.0,332,5780,5533,496,V420,40390,2851,5990,570,41071,4280,4443.0,5849.0,-1,-1,-1,-1,7,7,10.0,2.0,0.0,0
1,BENE11017,CLM31237,2008-12-25,2009-01-08,PRV54750,14000,PHY314656,PHY426644,-1,5939,1068.0,661,1889,41071,5990,5601,4588,5845,4549,29570,34831,-1,5551.0,-1,-1,-1,-1,-1,14,14,9.0,1.0,0.0,0
2,BENE11026,CLM78930,2009-12-09,2009-12-13,PRV53758,2000,PHY349495,-1,-1,4019,1068.0,241,4010,78791,60000,41401,V1254,4372,78650,7813,4254,-1,-1,-1,-1,-1,-1,-1,4,4,9.0,0.0,0.0,0
3,BENE11031,CLM56810,2009-06-23,2009-07-06,PRV55825,16000,PHY429538,PHY371893,-1,8208,1068.0,564,8208,4168,920,5990,40391,2859,4254,41400,5849,41401,8152.0,3320.0,-1,-1,-1,-1,13,13,10.0,2.0,0.0,0
4,BENE11085,CLM34625,2009-01-20,2009-01-31,PRV52338,19000,PHY397161,-1,-1,4279,1068.0,880,29654,V142,78702,30503,V140,V4582,V6109,7242,-1,-1,-1,-1,-1,-1,-1,-1,11,11,8.0,0.0,0.0,0


In [67]:
inp_train.shape, outp_train.shape

((9551, 34), (517737, 33))

In [68]:
ip_op_train = pd.concat([inp_train[inp_imp_cols], outp_train[outp_imp_cols]])
ip_op_train.shape

(527288, 11)

In [69]:
ip_op_train.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag
0,BENE11014,CLM67387,PRV57070,9000,1068.0,7,7,10.0,2.0,0.0,0
1,BENE11017,CLM31237,PRV54750,14000,1068.0,14,14,9.0,1.0,0.0,0
2,BENE11026,CLM78930,PRV53758,2000,1068.0,4,4,9.0,0.0,0.0,0
3,BENE11031,CLM56810,PRV55825,16000,1068.0,13,13,10.0,2.0,0.0,0
4,BENE11085,CLM34625,PRV52338,19000,1068.0,11,11,8.0,0.0,0.0,0


In [70]:
ip_op_train.columns

Index(['BeneID', 'ClaimID', 'Provider', 'InscClaimAmtReimbursed',
       'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag'],
      dtype='object')

In [71]:
label_train.columns, label_train.shape

(Index(['Provider', 'PotentialFraud'], dtype='object'), (5410, 2))

In [72]:
ip_op_train = pd.merge(left=ip_op_train, right=label_train, left_on='Provider', right_on= 'Provider', how='inner')
ip_op_train.shape

(517737, 12)

In [73]:
ip_op_train['PotentialFraud'].value_counts()

,count
PotentialFraud,
0,328343
1,189394


In [75]:
train_df = pd.merge(left=ip_op_train, right=bene_train, left_on='BeneID', right_on='BeneID',how='inner')
train_df.shape

(517737, 40)

In [79]:
train_df.to_csv('/content/sample_data/training_data.csv')

In [80]:
train_df.isna().sum()

,0
BeneID,0
ClaimID,0
Provider,0
InscClaimAmtReimbursed,0
DeductibleAmtPaid,0
ClaimPeriod,0
TimeInHptal,0
Diagnosis Count,0
Procedures Count,0
SamePhysician,212732
